# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala _K_-Means

In [ ]:
import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.SimpleRealClusterizable
import clustering4ever.scala.vectorizables.RealVector

import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.SimpleRealClusterizable
import clustering4ever.scala.vectorizables.RealVector


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2018-10-27 20:22:14--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv’

     0K .......                                               100% 77.8K=0.1s

2018-10-27 20:22:15 (77.8 KB/s) - ‘/tmp/aggregation.csv’ saved [8063/8063]

:sh: Scheme missing.
--2018-10-27 20:22:15--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2018-10-27 20:22:15--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels’

     0K .                                                     100% 61

## Import data as ParArray[ArrayBuffer[Double]]

In [ ]:
val datasetSize = 1000000
val dim = 3
val path = "/tmp/aggregation.csv"
val useAggregationDS = true

val rawData = if( useAggregationDS ) Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toBuffer
  else mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble))

val parData = rawData.zipWithIndex.par.map{ case (v, id) => new SimpleRealClusterizable(id.toLong, new RealVector(v)) }

val groundTruePath = "/tmp/labels"

datasetSize: Int = 1000000
dim: Int = 3
path: String = /tmp/aggregation.csv
useAggregationDS: Boolean = true
rawData: scala.collection.mutable.Buffer[scala.collection.mutable.ArrayBuffer[Double]] = ArrayBuffer(ArrayBuffer(15.55, 28.65), ArrayBuffer(14.9, 27.55), ArrayBuffer(14.45, 28.35), ArrayBuffer(14.15, 28.8), ArrayBuffer(13.75, 28.05), ArrayBuffer(13.35, 28.45), ArrayBuffer(13.0, 29.15), ArrayBuffer(13.45, 27.5), ArrayBuffer(13.6, 26.5), ArrayBuffer(12.8, 27.35), ArrayBuffer(12.4, 27.85), ArrayBuffer(12.3, 28.4), ArrayBuffer(12.2, 28.65), ArrayBuffer(13.4, 25.1), ArrayBuffer(12.95, 25.95), ArrayBuffer(12.9, 26.5), ArrayBuffer(11.85, 27.0), ArrayBuffer(11.35, 28.0), ArrayBuffer(11.15, 28.7), ArrayBuffer(11.25, 27.4), ArrayBuffer(10.75, 27.7), ArrayBuffer(10.5, 28.35), ArrayBuffer(9....

## Parameters 

In [ ]:
val k = 7
val iterMax = 50
val epsilon = 0.001
// True defines the true Euclidean with square root, set it to false to avoid the square
// val metric0 = new Euclidean[Array[Double]](true)
val metric1 = new Euclidean[mutable.ArrayBuffer[Double]](false)
// Minkowski(p) where p is the Minkowski parameter
val metric2 = new Minkowski[mutable.ArrayBuffer[Double]](2)

k: Int = 7
iterMax: Int = 50
epsilon: Double = 0.001
metric1: clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean distance without root applied
metric2: clustering4ever.math.distances.scalar.Minkowski[scala.collection.mutable.ArrayBuffer[Double]] = clustering4ever.math.distances.scalar.Minkowski@28ee503c


## Run and measure the algorithm time

In [ ]:
val data = parData.seq

val t1 = System.nanoTime

val model = KMeans.run(parData, k, epsilon, iterMax, metric1)

val t2 = System.nanoTime

(t2 - t1) / 1000000000D

data: scala.collection.mutable.Seq[clustering4ever.scala.clusterizables.SimpleRealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ArraySeq(SimpleRealClusterizable(0,clustering4ever.scala.vectorizables.RealVector@14ff5b83,None,None), SimpleRealClusterizable(1,clustering4ever.scala.vectorizables.RealVector@3acd1710,None,None), SimpleRealClusterizable(2,clustering4ever.scala.vectorizables.RealVector@77ef2cb5,None,None), SimpleRealClusterizable(3,clustering4ever.scala.vectorizables.RealVector@2f3ba1e5,None,None), SimpleRealClusterizable(4,clustering4ever.scala.vectorizables.RealVector@4176c4c2,None,None), SimpleRealClusterizable(5,clustering4ever.scala.vectorizables.RealVector@13512e6b,None,None), SimpleRealClusterizable(6,clu...

0.145388155

# Compare algorithm between ParArray and Seq container

In [ ]:
val t1 = System.nanoTime
KMeans.run(parData, k, epsilon, iterMax, metric1)
val t2 = System.nanoTime
val seqData = parData.seq
val t3 = System.nanoTime
KMeans.run(seqData, k, epsilon, iterMax, metric1)
val t4 = System.nanoTime



(t4 - t3).toDouble / (t2 - t1)

t1: Long = 4805462160376
t2: Long = 4805493509345
seqData: scala.collection.mutable.Seq[clustering4ever.scala.clusterizables.SimpleRealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ArraySeq(SimpleRealClusterizable(0,clustering4ever.scala.vectorizables.RealVector@14ff5b83,None,None), SimpleRealClusterizable(1,clustering4ever.scala.vectorizables.RealVector@3acd1710,None,None), SimpleRealClusterizable(2,clustering4ever.scala.vectorizables.RealVector@77ef2cb5,None,None), SimpleRealClusterizable(3,clustering4ever.scala.vectorizables.RealVector@2f3ba1e5,None,None), SimpleRealClusterizable(4,clustering4ever.scala.vectorizables.RealVector@4176c4c2,None,None), SimpleRealClusterizable(5,clustering4ever.scala.vectorizables.RealVect...

0.3341173038258451

## Different ways to apply model to datasets

In [ ]:
val clusterized11 = rawData.map( v => (model.centerPredict(v), v) )
val clusterized12 = model.centerPredict(rawData)

val clusterized21 = parData.map( clustbl => (model.centerPredict(clustbl.vector), clustbl) )
val clusterized22 = model.centerPredict(parData)

clusterized11: scala.collection.mutable.Buffer[(model.ClusterID, scala.collection.mutable.ArrayBuffer[Double])] = ArrayBuffer((1,ArrayBuffer(15.55, 28.65)), (1,ArrayBuffer(14.9, 27.55)), (1,ArrayBuffer(14.45, 28.35)), (1,ArrayBuffer(14.15, 28.8)), (1,ArrayBuffer(13.75, 28.05)), (1,ArrayBuffer(13.35, 28.45)), (1,ArrayBuffer(13.0, 29.15)), (1,ArrayBuffer(13.45, 27.5)), (1,ArrayBuffer(13.6, 26.5)), (1,ArrayBuffer(12.8, 27.35)), (1,ArrayBuffer(12.4, 27.85)), (1,ArrayBuffer(12.3, 28.4)), (1,ArrayBuffer(12.2, 28.65)), (1,ArrayBuffer(13.4, 25.1)), (1,ArrayBuffer(12.95, 25.95)), (1,ArrayBuffer(12.9, 26.5)), (1,ArrayBuffer(11.85, 27.0)), (1,ArrayBuffer(11.35, 28.0)), (1,ArrayBuffer(11.15, 28.7)), (1,ArrayBuffer(11.25, 27.4)), (1,ArrayBuffer(10.75, 27.7)), (1,ArrayBuffer(10.5, 28.35)), (1,ArrayBu...

In [ ]:
clusterized22

res12: scala.collection.GenSeq[clustering4ever.scala.clusterizables.SimpleRealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ParArray(SimpleRealClusterizable(0,clustering4ever.scala.vectorizables.RealVector@14ff5b83,None,Some(1)), SimpleRealClusterizable(1,clustering4ever.scala.vectorizables.RealVector@3acd1710,None,Some(1)), SimpleRealClusterizable(2,clustering4ever.scala.vectorizables.RealVector@77ef2cb5,None,Some(1)), SimpleRealClusterizable(3,clustering4ever.scala.vectorizables.RealVector@2f3ba1e5,None,Some(1)), SimpleRealClusterizable(4,clustering4ever.scala.vectorizables.RealVector@4176c4c2,None,Some(1)), SimpleRealClusterizable(5,clustering4ever.scala.vectorizables.RealVector@13512e6b,None,Some(1)), SimpleRealClust...

ParArray(SimpleRealClusterizable(0,clustering4ever.scala.vectorizables.RealVector@14ff5b83,None,Some(1)), SimpleRealClusterizable(1,clustering4ever.scala.vectorizables.RealVector@3acd1710,None,Some(1)), SimpleRealClusterizable(2,clustering4ever.scala.vectorizables.RealVector@77ef2cb5,None,Some(1)), SimpleRealClusterizable(3,clustering4ever.scala.vectorizables.RealVector@2f3ba1e5,None,Some(1)), SimpleRealClusterizable(4,clustering4ever.scala.vectorizables.RealVector@4176c4c2,None,Some(1)), SimpleRealClusterizable(5,clustering4ever.scala.vectorizables.RealVector@13512e6b,None,Some(1)), SimpleRealClusterizable(6,clustering4ever.scala.vectorizables.RealVector@1dd52230,None,Some(1)), SimpleRealClusterizable(7,clustering4ever.scala.vectorizables.RealVector@6f139ce6,None,Some(1)), SimpleRealClusterizable(8,clustering4ever.scala.vectorizables.RealVector@18b40512,None,Some(1)), SimpleRealClusterizable(9,clustering4ever.scala.vectorizables.RealVector@20c5219,None,Some(1)), SimpleRealClusterizable(10,clustering4ever.scala.vectorizables.RealVector@367c24ca,None,Some(1)), SimpleRealClusterizable(11,clustering4ever.scala.vectorizables.RealVector@45fb7a4f,None,Some(1)), SimpleRealClusterizable(12,clustering4ever.scala.vectorizables.RealVector@7ddf7b77,None,Some(1)), SimpleRealClusterizable(13,clustering4ever.scala.vectorizables.RealVector@1da3624f,None,Some(1)), SimpleRealClusterizable(14,clustering4ever.scala.vectorizables.RealVector@4a2e4f45,None,Some(1)), SimpleRealClusterizable(15,clustering4ever.scala.vectorizables.RealVector@54997d4d,None,Some(1)), SimpleRealClusterizable(16,clustering4ever.scala.vectorizables.RealVector@529bb06a,None,Some(1)), SimpleRealClusterizable(17,clustering4ever.scala.vectorizables.RealVector@542c116c,None,Some(1)), SimpleRealClusterizable(18,clustering4ever.scala.vectorizables.RealVector@6cc028cb,None,Some(1)), SimpleRealClusterizable(19,clustering4ever.scala.vectorizables.RealVector@732c530b,None,Some(1)), SimpleRealClusterizable(20,clustering4ever.scala.vectorizables.RealVector@7cbba5e4,None,Some(1)), SimpleRealClusterizable(21,clustering4ever.scala.vectorizables.RealVector@48286c6e,None,Some(1)), SimpleRealClusterizable(22,clustering4ever.scala.vectorizables.RealVector@254383d0,None,Some(1)), SimpleRealClusterizable(23,clustering4ever.scala.vectorizables.RealVector@34e63878,None,Some(1)), SimpleRealClusterizable(24,clustering4ever.scala.vectorizables.RealVector@15532f85,None,Some(1)), SimpleRealClusterizable(25,clustering4ever.scala.vectorizables.RealVector@1e937a5d,None,Some(1)), SimpleRealClusterizable(26,clustering4ever.scala.vectorizables.RealVector@27f7375c,None,Some(1)), SimpleRealClusterizable(27,clustering4ever.scala.vectorizables.RealVector@22d7fe22,None,Some(1)), SimpleRealClusterizable(28,clustering4ever.scala.vectorizables.RealVector@2e74c2fa,None,Some(1)), SimpleRealClusterizable(29,clustering4ever.scala.vectorizables.RealVector@2b3014fb,None,Some(1)), SimpleRealClusterizable(30,clustering4ever.scala.vectorizables.RealVector@35638b16,None,Some(1)), SimpleRealClusterizable(31,clustering4ever.scala.vectorizables.RealVector@28584576,None,Some(1)), SimpleRealClusterizable(32,clustering4ever.scala.vectorizables.RealVector@20b33195,None,Some(1)), SimpleRealClusterizable(33,clustering4ever.scala.vectorizables.RealVector@b8f7eb4,None,Some(1)), SimpleRealClusterizable(34,clustering4ever.scala.vectorizables.RealVector@229d7fe8,None,Some(1)), SimpleRealClusterizable(35,clustering4ever.scala.vectorizables.RealVector@4804fd38,None,Some(1)), SimpleRealClusterizable(36,clustering4ever.scala.vectorizables.RealVector@71024234,None,Some(1)), SimpleRealClusterizable(37,clustering4ever.scala.vectorizables.RealVector@108f87e,None,Some(1)), SimpleRealClusterizable(38,clustering4ever.scala.vectorizables.RealVector@48a3b588,None,Some(1)), SimpleRealClusterizable(39,clustering4ever.scala.vectorizables.RealVector@27a16c99,None,Some(1)), SimpleRealClusterizable(40,clustering4ever.scala.vectorizables.RealVector@445189bb,N

## Plot clustering results

In [ ]:
val rawData = clusterized22.map( rc => rc.vector.toArray ).toArray
val labels = clusterized22.map( rc => rc.clusterID.get ).toArray
plot(rawData, labels, '*', Palette.COLORS)

rawData: Array[Array[Double]] = Array(Array(15.55, 28.65), Array(14.9, 27.55), Array(14.45, 28.35), Array(14.15, 28.8), Array(13.75, 28.05), Array(13.35, 28.45), Array(13.0, 29.15), Array(13.45, 27.5), Array(13.6, 26.5), Array(12.8, 27.35), Array(12.4, 27.85), Array(12.3, 28.4), Array(12.2, 28.65), Array(13.4, 25.1), Array(12.95, 25.95), Array(12.9, 26.5), Array(11.85, 27.0), Array(11.35, 28.0), Array(11.15, 28.7), Array(11.25, 27.4), Array(10.75, 27.7), Array(10.5, 28.35), Array(9.65, 28.45), Array(10.25, 27.25), Array(10.75, 26.55), Array(11.7, 26.35), Array(11.6, 25.9), Array(11.9, 25.05), Array(12.6, 24.05), Array(11.9, 24.5), Array(11.1, 25.2), Array(10.55, 25.15), Array(10.05, 25.95), Array(9.35, 26.6), Array(9.3, 27.25), Array(9.2, 27.8), Array(7.5, 28.25), Array(8.55, 27.45), Ar...

Window(javax.swing.JFrame[frame0,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 1,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,37,1000x963,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x963,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])

## Inspect performance metrics

In [ ]:
val trueLabels = Source.fromFile(groundTruePath).getLines.map(_.toInt).toBuffer.toParArray

val (_, trueLabelsFrom0) = ExternalIndexes.prepareLabels(trueLabels)

val predicts = clusterized22.map(_.clusterID.get).toParArray

val nmi = ExternalIndexes.nmi(trueLabelsFrom0, predicts, SQRT)

nmi

trueLabels: scala.collection.parallel.mutable.ParArray[Int] = ParArray(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...

0.8423607670595896